In [1]:
import nltk
from nltk.wsd import lesk
from pywsd.similarity import max_similarity,similarity_by_path
from pywsd.lesk import simple_lesk, adapted_lesk, cosine_lesk,original_lesk

from nltk.tokenize import PunktSentenceTokenizer,sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet as wn
import string

import support.scrape as scrape


stop_words = set(stopwords.words('english')) 

def simpleFilter(sentence):
    filtered_sent = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))

    return filtered_sent

def listToString(inputSentList):
    return ' '.join(inputSentList)

def viewAllDefinitions(wordStr):
    resList = []
    for syn in wn.synsets(wordStr):
        resList.append(syn.definition())
    return resList

def topRelevSent(targetWord):
    return

def simpleFilter(sentence):
    filtered_sent = []
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(sentence)

    for w in words:
        if w not in stop_words:
            filtered_sent.append(lemmatizer.lemmatize(w))
    return listToString(filtered_sent)

def simpleFilterList(sentList):
    resList = []
    for sent in sentList:
        sent = sent.translate(str.maketrans('', '', string.punctuation))
        resList.append(simpleFilter(sent))
    return resList





sent = 'His card was the 8 of hearts'
ambiguous = 'heart'

sent = simpleFilter(sent)


print(sent)
cosine_lesk(sent,ambiguous).definition()



# original_lesk seems to be better

Warming up PyWSD (takes ~10 secs)... took 13.89805269241333 secs.


His card 8 heart


'a playing card in the major suit that has one or more red hearts on it'

In [2]:

        
viewAllDefinitions('bank')     


['sloping land (especially the slope beside a body of water)',
 'a financial institution that accepts deposits and channels the money into lending activities',
 'a long ridge or pile',
 'an arrangement of similar objects in a row or in tiers',
 'a supply or stock held in reserve for future use (especially in emergencies)',
 'the funds held by a gambling house or the dealer in some gambling games',
 'a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force',
 'a container (usually with a slot in the top) for keeping money at home',
 'a building in which the business of banking transacted',
 'a flight maneuver; aircraft tips laterally about its longitudinal axis (especially in turning)',
 'tip laterally',
 'enclose with a bank',
 'do business with a bank or keep an account at a bank',
 'act as the banker in a game or in gambling',
 'be in the banking business',
 'put into a bank account',
 'cover with ashes so to co

In [39]:
# nltk
sent = 'I went to the bank to deposit my money'
ambiguous = 'bank'
original_lesk(sent,ambiguous).definition()

'a financial institution that accepts deposits and channels the money into lending activities'

In [4]:
original_lesk('my cat likes to eat mice', 'cat').definition()

'feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats'

In [5]:
#sent = 'She is mean to me'
#sent = 'She mean me'
sent = 'What do you mean'

original_lesk(sent, 'mean').definition()
#simple_lesk(sent, 'mean').definition()

'mean or intend to express or convey'

In [6]:
# 'run'  'advice' , 'bat', 'project'  'fine'  'consoil'  'roll'  ''
sent = 'The rebel seized the opportunity to rebel'
adapted_lesk(sent, 'rebel').definition()

'break with established customs'

In [41]:
def seperateByDef(targetWord):
    # Returns a dictionary sorted by defintion
    sentList = scrape.scrape(targetWord)
    
    dictDef = {}
    for i,sent in enumerate(sentList):
        
        #defineSent = cosine_lesk(sent,targetWord).definition()
        defineSent = adapted_lesk(sent,targetWord).definition()        
        
        if defineSent not in dictDef:
            dictDef[defineSent] = [sent]
        else:
            dictDef[defineSent].append(sent)
    return dictDef

def printDictKeysVals(inputDict):
    for key in inputDict.keys():
        print('-------------------------------------------------------')
        print('Def: ', key)
        for value in inputDict[key]:
            print('        ', value)
    
def getDef(sent,targetWord):
    # Get defintion of word
    
    #defineSent = cosine_lesk(sent,targetWord).definition()
    #defineSent = adapted_lesk(sent,targetWord).definition() 
    defineSent = original_lesk(sent,targetWord).definition() 
    
    return defineSent

def printDefExampleSent(inputDict,wordDef):
    print('Definition: ')
    print(wordDef)
    print('\n\n\n')
    print('Example sentence:')
    
    
    
    if wordDef not in inputDict:
        print('No example sentences found')
        return
    
    for sent in inputDict[wordDef]:
        print('-',sent)
        print('')
        
        
def countDefFreq(inputDict):
    resDict = {}
    for key in inputDict.keys():
        resDict[key] = len(inputDict[key])
    return sorted(resDict.items(), key=lambda kv: kv[1])

targetWord = 'bank'
#testSent = 'He does not have money in his bank account'
testSent = 'He ran out of money to deposit into his bank account'



dictDef = seperateByDef(targetWord)
wordDef = getDef(testSent,targetWord)

printDefExampleSent(dictDef,wordDef)
#printDictKeysVals(dictDef)



# Print out frequency of each definiton used
#print(countDefFreq(dictDef))


Definition: 
put into a bank account




Example sentence:
- Her bank account was rarely over two hundred.

- An overseas bank account in the company name contained a balance of one million dollars as startup funds for us to secure quarters for the enterprise.

- After breakfast, she closed her account at the bank and turned in her resignation.

- About 80,000 went in payments on all the estates to the Land Bank, about 30,000 went for the upkeep of the estate near Moscow, the town house, and the allowance to the three princesses; about 15,000 was given in pensions and the same amount for asylums; 150,000 alimony was sent to the countess; about 70,000 went for interest on debts.

- In less than a week they would be married, but somehow it seemed a violation of his bank account.

- I cleaned out our bank accounts and wrapped the withdrawal slips in the same pair of shorts and pinned it to his pillow with a steak knife and took off.

- Dean's feelings were mixed about Pumpkin's visit but 

In [22]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AffinityPropagation
from nltk.stem.snowball import SnowballStemmer

punctuation_map = dict((ord(char), None) for char in string.punctuation)
stemmer = SnowballStemmer("english")

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def get_clusters(sentences):
    tf_idf_matrix = vectorizer.fit_transform(sentences)
    similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
    affinity_propagation = AffinityPropagation(damping=0.5)
    affinity_propagation.fit(similarity_matrix)
    print(similarity_matrix.shape)

    labels = affinity_propagation.labels_
    cluster_centers = affinity_propagation.cluster_centers_indices_

    tagged_sentences = zip(sentences, labels)
    clusters = {}

    for sentence, cluster_id in tagged_sentences:
        clusters.setdefault(sentences[cluster_centers[cluster_id]], []).append(sentence)

    return clusters, affinity_propagation

sentences = scrape.scrape('cat')


#sentences = simpleFilterList(sentences)

clusters, affinity_propagation = get_clusters(sentences)
print(len(clusters))
for cluster in clusters:
    print('--------------------------------------')
    print(cluster, ':')
    for element in clusters[cluster]:
        print('  - ', element)

        
        
tf_idf_matrix = vectorizer.fit_transform(['The cat is running away from the dog'])
similarity_matrix = (tf_idf_matrix * tf_idf_matrix.T).A
print(affinity_propagation.predict(similarity_matrix))

(91, 91)
20
--------------------------------------
"I like your cat," he said. :
  -  "You look like the cat that swallowed the canary," he said, giving her a puzzled look.
  -  "First time I ever saw a pink cat," said Zeb.
  -  Dusty said I have to leave the cat with you.
  -  "I've never had a cat," he said, excited.
  -  "I like your cat," he said.
--------------------------------------
I didn't want the cat to stay all alone at your condo. :
  -  I think the cat wants dessert!
  -  I didn't want the cat to stay all alone at your condo.
  -  Can I just stay here and guard your house and cat for you?
--------------------------------------
To her surprise, the cat gave him no resistance. :
  -  I'm in no mood to watch a cat fight tonight.
  -  Once I was surprised to see a cat walking along the stony shore of the pond, for they rarely wander so far from home.
  -  Dean went to cat and girl and gave them both a long hug.
  -  He returned to the house to await Darian, not surprised to s

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1 while Y.shape[1] == 91